source API URL : "https://geocoding-api.open-meteo.com/v1/search?name=kovilpatti&count=10&language=en&format=json"

JSON Target File Path : "abfss://bronze@datalakestorageaccountname.dfs.core.windows.net/geo-location/"

In [0]:
geoLocationSourceAPIURL = "https://geocoding-api.open-meteo.com/v1/search?name=kovilpatti&count=10&language=en&format=json"

geoLocationSourceAPIBaseURL = "https://geocoding-api.open-meteo.com/v1/search?name="
geoLocationSourceAPIURLOptions = "&count=10&language=en&format=json"

geoLocationSinkLayerName = 'bronze'
geoLocationSinkStorageAccountName = 'adlsdatalakedeve'
geoLocationSinkFolderName = 'geo-location'

geoLocationSinkFolderPath = f"abfss://{geoLocationSinkLayerName}@{geoLocationSinkStorageAccountName}.dfs.core.windows.net/{geoLocationSinkFolderName}"

In [0]:
import requests
import json
import pandas as pds

In [0]:
#geoLocationAPIResponse = requests.get(geoLocationSourceAPIURL).json()
#geoLocationPandasDF = pds.DataFrame(geoLocationAPIResponse)
#geoLocationSparkDF = spark.createDataFrame(geoLocationPandasDF)

In [0]:
dailyPricingMarketNamesDF = spark.sql("SELECT MARKET_NAME from pricing_analytics.gold.reporting_dim_market_gold ")

In [0]:
from pyspark.sql.types import *
marketNames = [dailyPricingMarketNames["MARKET_NAME"] for dailyPricingMarketNames in dailyPricingMarketNamesDF.collect()]


geoLocationAPIResponseList = []
for marketName in marketNames:
  
  geoLocationSourceAPIURL = f"{geoLocationSourceAPIBaseURL}{marketName}{geoLocationSourceAPIURLOptions}"
  geoLocationAPIResponse = requests.get(geoLocationSourceAPIURL).json()
  
  
  if isinstance(geoLocationAPIResponse, dict):
      geoLocationAPIResponseList.append(geoLocationAPIResponse)    


geoLocationSparkRDD =sc.parallelize(geoLocationAPIResponseList)

geoLocationSparkDF = spark.read.json(geoLocationSparkRDD)

(geoLocationSparkDF
.filter("results.admin1 IS NOT NULL")##api fault
.write
.mode("overwrite")
.json(geoLocationSinkFolderPath))
      


In [0]:
from pyspark.sql.functions import col, array_contains
geoLocationBronzeDF = (spark
                       .read
                                             .json(geoLocationSinkFolderPath))
                       

display(geoLocationBronzeDF)